# Feature Engineering
- O processo de **extração, transformação, criação e seleção de recursos (features) dos dados**
    - **"Feature"**: em resumo, é qualquer atributo nos dados capaz de **diferenciar alguma coisa** 
        - Por exemplo, para diferenciar uma banana de uma maça podemos usar as features: **formato, cor, textura**
    <img src="https://drive.google.com/uc?id=1w2PykpRyPhDF7GLkEsEi46q44ZrklZRe" style='width: 450px;' /><br>
            - Tradução: atributo ou aspecto **distintivo (capaz de distinguir)** de algo
    <br><br>
    - Como vantagens:
        - Melhora o resultado do modelo
        - Viabiliza o uso de determinadas informações
        - Reduz esforço computacional
        - Aumenta a **generalização**
    - Consome grande parte do tempo do projeto
<br><br>  
- Vamos usar o dataset obtido no módulo de Data Cleaning, disponível em `train2.csv` no link abaixo:
    - https://drive.google.com/file/d/1Mlqxk_DFbeLghUd4LXOjbQXTPuLWKSIl/view?usp=share_link

In [1]:
# Importando o pandas
import pandas as pd

In [2]:
# Importando os dados do titanic após as etapas de Data Cleaning
titanic = pd.read_csv('train2.csv')

In [3]:
# Visualizando a tabela
titanic.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titulos
0,0,3,male,22.0,1,0,7.2500,S,Mr
1,1,1,female,38.0,1,0,71.2833,C,Mrs


- **Se tentarmos usar a árvore de classificação**
    - https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

In [4]:
# Importando o DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

In [5]:
# Criando o classificador
clf = DecisionTreeClassifier(random_state=0)

In [6]:
# Separando X e Y
X = titanic.drop('Survived',axis=1)
y = titanic.Survived

In [7]:
# Fazendo o fit com os dados
clf = clf.fit(X,y)

ValueError: could not convert string to float: 'male'

## <font color='white'> Tratando variáveis de texto </font>
- **Grande parte dos modelos não é capaz de receber valores de texto e devemos transformar esses valores em numéricos**

- **Vamos começar tratando os problemas que inviabilizam de usarmos os dados no nosso modelo**
    - `ValueError: could not convert string to float: 'male'`

In [8]:
# Verificando os valores na coluna Sex
titanic.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

- Como temos apenas 2 valores, podemos considerar que ou o valor é `male` ou não é
    - **Podemos usar 1 quando o valor for `male` e 0 quando não for**
    - Para fazer isso podemos usar a <font color='white'>**lambda function para verificar se o valor é igual ou não a 'male'**</font>
    - É importante entendermos com o negócio se vamos receber novos valores além desses e, caso isso aconteça, pensarmos na melhor maneira de tratar esse valor

In [9]:
# Verificando se o valor na coluna é "male"
titanic['Is_Male'] = titanic.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [10]:
# Verificando a base com essa nova coluna
titanic.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titulos,Is_Male
0,0,3,male,22.0,1,0,7.2500,S,Mr,1
1,1,1,female,38.0,1,0,71.2833,C,Mrs,0
2,1,3,female,26.0,0,0,7.9250,S,Miss,0
3,1,1,female,35.0,1,0,53.1000,S,Mrs,0
4,0,3,male,35.0,0,0,8.0500,S,Mr,1


In [11]:
# Podemos inclusive fazer um groupby para verificar se todos os dados da nova coluna estão corretos
titanic[['Sex','Is_Male']].value_counts()

Sex     Is_Male
male    1          577
female  0          314
dtype: int64

- Já que a informação nas colunas `Is_Male` e `Sex`estão nos dando **a mesma informação** sobre os dados, **não é necessário manter as duas**

In [12]:
# Excluindo a coluna "Sex"
titanic = titanic.drop('Sex',axis=1)

In [13]:
### Novamente tentando fazer o fit com os dados
# Separando X e Y
X = titanic.drop('Survived',axis=1)
y = titanic.Survived

# Fazendo o fit
clf = clf.fit(X,y)

ValueError: could not convert string to float: 'S'

In [14]:
titanic.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

- **Ainda temos o mesmo problema de tentar passar uma string para o nosso modelo, mas agora temos mais de 2 categorias (não podemos criar uma coluna apenas com 1 ou 0)**
    - `ValueError: could not convert string to float: 'S'`
    - Como <font color='white'>**não existe uma relação de ordem entre os valores**</font>, podemos utilizar o <font color='white'>**One Hot Encoder**</font>
        - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [15]:
# Importando
from sklearn.preprocessing import OneHotEncoder

In [16]:
# Utilizando o OneHotEncoder
ohe = OneHotEncoder()

In [17]:
# Fazendo o fit com os dados
ohe = ohe.fit(X[['Embarked']])

In [19]:
# Fazendo a transformação da coluna Embarked
ohe.transform(X[['Embarked']]).toarray()

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [22]:
# Podemos transformar esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(X[['Embarked']]).toarray())
ohe_df.head(2)

,0,1,2
0,0.0,0.0,1.0
1,1.0,0.0,0.0


In [32]:
# Agora podemos utilizar o categories_ para nomear essas colunas
ohe_df.columns = ohe.get_feature_names_out()

In [33]:
# Visualizando esse DF
ohe_df.head(3)

,Embarked_C,Embarked_Q,Embarked_S
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0


In [34]:
# Visualizando o dataset titanic
titanic.head(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Titulos,Is_Male
0,0,3,22.0,1,0,7.2500,S,Mr,1
1,1,1,38.0,1,0,71.2833,C,Mrs,0
2,1,3,26.0,0,0,7.9250,S,Miss,0


In [ ]:
# Unindo essas duas informações

In [ ]:
# Visualizando esses dados

In [ ]:
# E então excluindo as duas colunas que usamos para fazer o OHE

In [ ]:
# Visualizando novamente

In [ ]:
# Novamente tentando fazer o fit com os dados
# Separando X e Y

# Fazendo o fit